# Installazioni

In [ ]:
!pip install optuna

# Imports

In [ ]:
import optuna
import numpy as np
import pandas as pd
import tensorflow as tf
import joblib

from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    make_scorer,
)
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler, MaxAbsScaler
from sklearn.utils import compute_class_weight
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split


from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import BatchNormalization, Dense, Dropout
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

# Rimozione dei warnings

In [ ]:
# Remove all warnings
import warnings
warnings.filterwarnings("ignore")
import os
os.environ["PYTHONWARNINGS"] = "ignore" # Also affect subprocesses

# Costanti

In [ ]:
LEARNING_RATE = 0.0001
EPOCHS = 200
RANDOM_STATE = 3993
tf.random.set_seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)

early_stopping = EarlyStopping(
    monitor="binary_accuracy", 
    verbose=1,
    patience=20,
    mode="max",
    restore_best_weights=True,
)

# Modello di rete neurale
Implementiamo un semplice modello di rete neurale, con una serie di iperparametri:


1.   numero di unità nel primo layer nascosto, viene poi decrementato per ogni successivo.
2.   numero di layer nascosti.
3.   funzione di attivazione dei layer nascosti.
4.   valore del dropout nel primo layer nascosto, viene poi decrementato per ogni successivo.
5.   booleano che indica se la batch normalization sia attiva nei layer nascosti.
6.   booleano che indica se l'activity regularization sia attiva nei layer nascosti.

Utilizziamo **Adam** come ottimizzatore con  un learning rate pari a $10^{-4}$, **softmax** come funzione di attivazione del layer di output e come loss la **binary crossentropy**, affiancata alla **binary accuracy**, come metrica.
Un altra importante implementazione è l'utilizzo del corretto inizializzatore dei pesi rispetto alla funzione di attivazione inserita come iperparametro.

In [ ]:
def deep_dense_nn(max_hidden_units: int,
                  hidden_layers: int = 1,
                  hidden_activation: str = 'relu',
                  max_dropout_rate: float = 0.5,
                  batch_norm: bool = False,
                  activity_regularizer: bool = False,
                 ):
    output_units = 2
    output_activation = 'softmax'
    
    # Define the correct kernel initialization for the selected activation function
    act_kinit_dict = {'relu': 'he_uniform', 'selu': 'lecun_normal', 'elu': 'he_uniform', 'swish': 'he_uniform'}
    kernel_intializer = act_kinit_dict[hidden_activation] if hidden_activation in act_kinit_dict else 'glorot_uniform'
    
    # Create the list of layers
    layers = list()
       
    for i in range(1, hidden_layers + 1):        
        # Add dense layer
        layers.append(Dense(units=int(max_hidden_units/i) + output_units,
                                activation=hidden_activation,
                                kernel_initializer=kernel_intializer,
                                activity_regularizer=l2(1e-5) if activity_regularizer else None,
                               ))
        # Add batch normalization if it is setted in params
        if batch_norm:
            layers.append(BatchNormalization())
        # Add Dropout
        layers.append(Dropout(max_dropout_rate/i))
        
    # Extend with last part of the layers
    layers.append(Dense(output_units, activation=output_activation))
    
#     print(layers)
    
    # Create sequential model
    model = Sequential(layers)
    optimizer = Adam(learning_rate=LEARNING_RATE)
    metrics = ["binary_accuracy"]
    # Compile the model
    model.compile(loss = "binary_crossentropy",
                  optimizer=optimizer,
                  metrics=metrics)
    
    #model.summary()
    
    return model

# Caricamento del dataset e preprocessing 
load del dataset e trasformazione della feature da **Good** a **1** e da **Disappointing** a **0**, in modo tale che il modello possa lavorare in maniera corretta

In [ ]:
url = 'https://raw.githubusercontent.com/serivan/mldmlab/master/Datasets/Kaggle2020/train.csv'
training_set = pd.read_csv(url)
for index in training_set.index:
    if training_set.loc[index,"Quality"]=="Good":
        training_set.loc[index,"Quality"] = 1
    if training_set.loc[index,"Quality"]=="Disappointing":
        training_set.loc[index,"Quality"] = 0

In [ ]:
training_set

,Id,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,Quality
0,3940,6.4,0.39,0.21,1.2,41.00,35.0,136.0,0.99225,3.15,0.46,10.2,0
1,1655,7.5,305.00,0.40,18.9,59.00,44.0,170.0,1.00000,2.99,0.46,9.0,0
2,1867,6.3,0.28,0.30,3.1,39.00,24.0,115.0,0.99420,3.05,0.43,8.6,0
3,4476,7.4,0.18,0.30,10.4,45.00,44.0,174.0,0.99660,3.11,0.57,9.7,1
4,453,6.9,0.20,0.36,1.5,31.00,38.0,147.0,0.99310,3.35,0.56,11.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3484,2768,6.9,0.14,0.38,1.0,41.00,22.0,81.0,0.99043,3.03,0.54,11.4,1
3485,4347,9.0,0.20,0.33,3.5,49.00,10.0,40.0,0.99440,3.14,0.36,9.8,1
3486,1870,7.6,0.29,0.26,6.5,42.00,32.0,160.0,0.99440,3.14,0.47,10.7,0
3487,613,7.5,0.17,0.32,1.7,0.04,51.0,148.0,0.99160,3.21,0.44,11.5,1


Spezziamo il data set in X e Y.
Poi eseguiamo lo split in traininig e testing, in modo tale da poter aver un riscontro rispetto a quello che stiamo svolgendo.

In [ ]:
# creo X e Y eseguendo il drop di id e target
cols_to_drop = ["Id", "Quality"]
X_train = training_set.drop(columns=cols_to_drop)
# setta la colonna target
y_train = training_set["Quality"]
y_train_cat = to_categorical(y_train)
X_train = X_train.fillna(np.nan)
X_train = X_train.astype(np.float32)

X_train, X_val, y_train_cat, y_val_cat = train_test_split(
    X_train, y_train_cat, test_size=0.2, random_state=RANDOM_STATE, stratify=y_train_cat,
)


In [ ]:
rounded_labels = np.argmax(y_train_cat, axis=1)
rounded_labels2 = np.argmax(y_val_cat, axis=1)

# Sbilanciamento delle classi
Possiamo vedere come il dataset risulta sbilanciato, il numero dei vini buoni è quasi il doppio rispetto a quelli non soddisfacenti, per questo motivo per equilibrare le classi è possibile seguire differenti strategie: downsampling, upsampling oppure l'assegnamento dei pesi alle classi. La strategia scelta per questo esempio è l'ultimo metodo.

In [ ]:
print("Campioni totali nel training set: "+str(training_set.shape[0]))
numbers_good = training_set[training_set["Quality"]==1].shape[0]
numbers_disappointing = training_set[training_set["Quality"]==0].shape[0]
print("Numero dei vini buoni: " +str(numbers_good)+", numero dei vini non soddisfacenti: "+str(numbers_disappointing))

Campioni totali nel training set: 3489
Numero dei vini buoni: 2284, numero dei vini non soddisfacenti: 1205


In [ ]:
# calcolo i pesi derivanti da ogni classe
class_weight = compute_class_weight("balanced", classes=np.unique(y_train), y=y_train)
class_weight = {0: np.float32(class_weight[0]), 1: np.float32(class_weight[1])}
print("Peso per i vini non soddisfacenti: " +str(class_weight[0])+", peso per i vini buoni: "+str(class_weight[1]))

Peso per i vini non soddisfacenti: 1.4477178, peso per i vini buoni: 0.7637916


# Prova di una configurazione
Qualora, non avessimo nessuna strategia di scelta degli iperparametri, sarebbe necessario sceglierli a mano, come nell'esempio successivo. Nella pipeline, nel fit inseriamo il numero di epoche, l'early stopping, un ulteriore regolarizzazione e i pesi relativi alle classi. 

In [ ]:
model = deep_dense_nn(max_hidden_units = 300,
                  hidden_layers = 3,
                  hidden_activation = 'relu',
                  max_dropout_rate  = 0.25,
                  batch_norm = True,
                  activity_regularizer = True,
                 )
pipe = Pipeline([
        ('imputer', SimpleImputer()),
        ('scalers', StandardScaler()),
        ('model', model)
        ])
pipeline = pipe.fit(
    X_train,
    rounded_labels,
    model__epochs=EPOCHS,
    model__callbacks=[early_stopping],
    model__class_weight=class_weight,
    model__verbose=0, 
)

Restoring model weights from the end of the best epoch.
Epoch 00021: early stopping


In [ ]:
predictions = pipeline.predict(X_train)
predictions = np.argmax(predictions.round(), axis=1)
print("Accuracy sul training set: "+str(accuracy_score(rounded_labels, predictions) * 100))
cm_training = confusion_matrix(rounded_labels, predictions)
print("Confusion matrix sul training: ")
print(cm_training)

predictions = pipeline.predict(X_val)
predictions = np.argmax(predictions.round(), axis=1)
print("Accuracy sul validation set: "+str(accuracy_score(rounded_labels2, predictions) * 100))
cm_training = confusion_matrix(rounded_labels2, predictions)
print("Confusion matrix sul validation: ")
print(cm_training)

Accuracy sul training set: 49.91042637047653
Confusion matrix sul training: 
[[ 625  339]
 [1059  768]]
Accuracy sul validation set: 49.713467048710605
Confusion matrix sul validation: 
[[164  77]
 [274 183]]


# Optuna, ricerca degli iperparametri tramite ottimizzazione bayesiana
Come già enunciato nelle slides, utilizzeremo Optuna, per sfruttare questo framework dovremo definire una serie di passi necessari:


1.   Definire uno spazio degli iperparametri (tramite un dizionario in cui, per ogni iperparametro, porremo una distribuzione fra quelle offerte sul sito https://optuna.readthedocs.io/en/stable/reference/distributions.html)
2.   Definire una pipeline (la nostra rete neurale con il suo preprocessing)
3.   Definire una funzione di score (la funzione su cui valutiamo la nostra rete neurale)



In [ ]:
#1
SEARCH_PARAMS_N1 = dict(
    imputer__fill_value =optuna.distributions.CategoricalDistribution([-1,]),
    imputer__strategy = optuna.distributions.CategoricalDistribution(['mean','median','constant',]),
    imputer__missing_values = optuna.distributions.CategoricalDistribution([np.nan,]),
    scalers = optuna.distributions.CategoricalDistribution([StandardScaler(),RobustScaler(),MinMaxScaler(),MaxAbsScaler()]),
    model__hidden_activation=optuna.distributions.CategoricalDistribution(["relu","elu","tanh","swish","selu"]),
    model__hidden_layers=optuna.distributions.IntUniformDistribution(1,10),
    model__max_hidden_units=optuna.distributions.IntUniformDistribution(100,1000),
    model__max_dropout_rate=optuna.distributions.UniformDistribution(0,0.5),
    model__batch_norm=optuna.distributions.CategoricalDistribution(["True","False"]),
    model__activity_regularizer=optuna.distributions.CategoricalDistribution(["True","False"]),
    model__batch_size = optuna.distributions.CategoricalDistribution([8,16,32,64]),
)
#2
model = KerasClassifier(build_fn=deep_dense_nn, verbose=0,)
pipe = Pipeline([
        ('imputer', SimpleImputer()),
        ('scalers', StandardScaler()),
        ('model', model)
        ])
#3
scorer = make_scorer(accuracy_score)

Una volta definiti i 3 passi necessari per il funzionamento di Optuna andiamo ora a definire l'ottimizzatore:
1.   il numero di prove da effettuare
2.   il grado di parallelizzazione(meglio porre 1)
3.   il refit tramite l'intero data set una volta scovato il modello migliore
4.   la ten cross fold validation per una migliore regolarizzazione

Una volta definito è sufficiente poi eseguire il train, come quando svolgiamo un semplice modello di rete neurale.
 
 
 

In [ ]:

grid = optuna.integration.OptunaSearchCV(
    estimator=pipe,
    param_distributions=SEARCH_PARAMS_N1,
    n_trials =3,
    scoring=scorer,
    n_jobs=1,
    refit=True,
    cv=10,
    verbose=1,
    random_state=RANDOM_STATE,
)
grid_result = grid.fit(
    X_train,
    rounded_labels,
    model__epochs=EPOCHS,
    model__callbacks=[early_stopping],
    model__class_weight=class_weight,
    model__verbose=0,
)

[I 2020-11-04 12:41:56,392] A new study created in memory with name: no-name-6e018699-460f-4d3d-ab23-904371f1d7b1
[I 2020-11-04 12:41:56,393] Searching the best hyperparameters using 2791 samples...


Restoring model weights from the end of the best epoch.
Epoch 00021: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00021: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00021: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00021: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00021: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00021: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00021: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00021: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00021: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00021: early stopping


[I 2020-11-04 12:45:08,995] Trial 0 finished with value: 0.5230849974398362 and parameters: {'imputer__fill_value': -1, 'imputer__strategy': 'median', 'imputer__missing_values': nan, 'scalers': MaxAbsScaler(copy=True), 'model__hidden_activation': 'swish', 'model__hidden_layers': 7, 'model__max_hidden_units': 460, 'model__max_dropout_rate': 0.3486766288315916, 'model__batch_norm': 'False', 'model__activity_regularizer': 'False', 'model__batch_size': 16}. Best is trial 0 with value: 0.5230849974398362.


Restoring model weights from the end of the best epoch.
Epoch 00021: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00021: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00021: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00021: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00021: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00021: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00021: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00021: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00021: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00021: early stopping


[I 2020-11-04 12:51:30,302] Trial 1 finished with value: 0.4994585253456222 and parameters: {'imputer__fill_value': -1, 'imputer__strategy': 'median', 'imputer__missing_values': nan, 'scalers': StandardScaler(copy=True, with_mean=True, with_std=True), 'model__hidden_activation': 'tanh', 'model__hidden_layers': 10, 'model__max_hidden_units': 510, 'model__max_dropout_rate': 0.2864285769474616, 'model__batch_norm': 'False', 'model__activity_regularizer': 'False', 'model__batch_size': 8}. Best is trial 0 with value: 0.5230849974398362.


Restoring model weights from the end of the best epoch.
Epoch 00021: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00021: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00021: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00021: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00021: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00021: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00021: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00021: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00021: early stopping
Restoring model weights from the end of the best epoch.
Epoch 00021: early stopping


[I 2020-11-04 12:57:01,296] Trial 2 finished with value: 0.5579403481822837 and parameters: {'imputer__fill_value': -1, 'imputer__strategy': 'mean', 'imputer__missing_values': nan, 'scalers': MinMaxScaler(copy=True, feature_range=(0, 1)), 'model__hidden_activation': 'swish', 'model__hidden_layers': 7, 'model__max_hidden_units': 507, 'model__max_dropout_rate': 0.4988168283845638, 'model__batch_norm': 'False', 'model__activity_regularizer': 'True', 'model__batch_size': 8}. Best is trial 2 with value: 0.5579403481822837.
[I 2020-11-04 12:57:01,297] Finished hyperparemeter search!
[I 2020-11-04 12:57:01,303] Refitting the estimator using 2791 samples...
[I 2020-11-04 12:57:42,389] Finished refitting! (elapsed time: 41.080 sec.)


Restoring model weights from the end of the best epoch.
Epoch 00021: early stopping


In [ ]:
grid_result_PD = pd.DataFrame(grid_result.study_.trials_dataframe())
grid_result_PD =grid_result_PD.sort_values(by=['value'],ascending=False)
grid_result_PD

,number,value,datetime_start,datetime_complete,duration,params_imputer__fill_value,params_imputer__missing_values,params_imputer__strategy,params_model__activity_regularizer,params_model__batch_norm,params_model__batch_size,params_model__hidden_activation,params_model__hidden_layers,params_model__max_dropout_rate,params_model__max_hidden_units,params_scalers,user_attrs_mean_fit_time,user_attrs_mean_score_time,user_attrs_mean_test_score,user_attrs_split0_test_score,user_attrs_split1_test_score,user_attrs_split2_test_score,user_attrs_split3_test_score,user_attrs_split4_test_score,user_attrs_split5_test_score,user_attrs_split6_test_score,user_attrs_split7_test_score,user_attrs_split8_test_score,user_attrs_split9_test_score,user_attrs_std_fit_time,user_attrs_std_score_time,user_attrs_std_test_score,state
2,2,0.557940,2020-11-04 12:51:30.303276,2020-11-04 12:57:01.295064,0 days 00:05:30.991788,-1,NaN,mean,True,False,8,swish,7,0.498817,507,"MinMaxScaler(copy=True, feature_range=(0, 1))",32.671958,0.424189,0.557940,0.346429,0.584229,0.652330,0.698925,0.526882,0.655914,0.491039,0.333333,0.645161,0.645161,1.238024,0.043236,0.124466,COMPLETE
0,0,0.523085,2020-11-04 12:41:56.394435,2020-11-04 12:45:08.994799,0 days 00:03:12.600364,-1,NaN,median,False,False,16,swish,7,0.348677,460,MaxAbsScaler(copy=True),18.840849,0.415674,0.523085,0.592857,0.648746,0.666667,0.315412,0.329749,0.627240,0.426523,0.630824,0.659498,0.333333,0.761313,0.042572,0.144283,COMPLETE
1,1,0.499459,2020-11-04 12:45:08.996652,2020-11-04 12:51:30.301296,0 days 00:06:21.304644,-1,NaN,median,False,False,8,tanh,10,0.286429,510,"StandardScaler(copy=True, with_mean=True, with...",37.603524,0.523631,0.499459,0.510714,0.465950,0.498208,0.498208,0.444444,0.523297,0.534050,0.444444,0.526882,0.548387,2.014285,0.054916,0.034952,COMPLETE


In [ ]:
print("Index migliore: " +str(grid_result.best_trial_.number))
print("Parametri migliori: "+str(grid_result.best_params_))
print("Score migliore: "+str(grid_result.best_score_))
predictions = grid_result.best_estimator_.predict(X_train)
print("Accuracy sul training set: "+str(accuracy_score(rounded_labels, predictions) * 100))
cm_training = confusion_matrix(rounded_labels, predictions)
print("Confusion matrix sul training: ")
print(cm_training)
predictions = grid_result.best_estimator_.predict(X_val)
print("Accuracy sul testing set: " +str(accuracy_score(rounded_labels2, predictions) * 100))
cm_testing = confusion_matrix(rounded_labels2, predictions)
print("Confusion matrix sul testing: ")
print(cm_testing)

Index migliore: 2
Parametri migliori: {'imputer__fill_value': -1, 'imputer__strategy': 'mean', 'imputer__missing_values': nan, 'scalers': MinMaxScaler(copy=True, feature_range=(0, 1)), 'model__hidden_activation': 'swish', 'model__hidden_layers': 7, 'model__max_hidden_units': 507, 'model__max_dropout_rate': 0.4988168283845638, 'model__batch_norm': 'False', 'model__activity_regularizer': 'True', 'model__batch_size': 8}
Score migliore: 0.5579403481822837
Accuracy sul training set: 60.4442852024364
Confusion matrix sul training: 
[[  69  895]
 [ 209 1618]]
Accuracy sul testing set: 58.59598853868195
Confusion matrix sul testing: 
[[ 19 222]
 [ 67 390]]


# Salvataggio del CSV e dello studio

In [ ]:
#grid_result_PD.to_csv("./dataset_kaggle/grid_result_PD.csv")
#joblib.dump(grid_result.study_, "./dataset_kaggle/study.pkl")

# Visualizzazione dello studio tramite grafici

In [ ]:
study = joblib.load('study.pkl')
#study = grid_result.study_

In [ ]:
optuna.visualization.plot_optimization_history(study).show()

In [ ]:
optuna.visualization.plot_param_importances(study, params= [
                                                      'model__batch_size',
                                                      'model__hidden_activation',
                                                      'model__hidden_layers',
                                                      'model__max_dropout_rate',
                                                      'model__max_hidden_units',
                                                      'imputer__strategy'
                                                     ]).show()

In [ ]:
optuna.visualization.plot_slice(study, params= [
                                                      'model__batch_size',
                                                      'model__hidden_activation',
                                                      'model__hidden_layers',
                                                      'model__max_dropout_rate',
                                                      'model__max_hidden_units',
                                                      'imputer__strategy'
                                                     ]).show()

In [ ]:
optuna.visualization.plot_parallel_coordinate(study, params= [
                                                      'model__batch_size',
                                                      'model__hidden_activation',
                                                      'model__hidden_layers',
                                                      'model__max_dropout_rate',
                                                      'model__max_hidden_units',
                                                      'imputer__strategy'
                                                     ]).show()